# Load the model

In [1]:
from langchain_ollama import OllamaLLM, ChatOllama

# Use a valid model name that Ollama is serving locally.
model_name = "llama3.2:latest" # "qwen2.5:3b" # "phi3:mini"

# Instantiate the LLM with the specified model
llm = ChatOllama(
    model=model_name,
)

# Tool Calls with Pydantic

In [2]:
from pydantic import BaseModel, Field

class WeatherSchema(BaseModel):
    condition: str = Field(description="Weather condition such as sunny, rainy, cloudy")
    temperature: int = Field(description="Temperature value")
    unit: str = Field(description="Temperature unit such as fahrenheit or celsius")

weather_llm = llm.bind_tools(tools=[WeatherSchema])
response = weather_llm.invoke("It's sunny and 75 degrees")
# Returns: {"condition": "sunny", "temperature": 75, "unit": "fahrenheit"}
print(response)
response.pretty_print()

content='' additional_kwargs={} response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-12-31T14:36:07.367606102Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8306052440, 'load_duration': 4413848648, 'prompt_eval_count': 192, 'prompt_eval_duration': 1188324019, 'eval_count': 31, 'eval_duration': 2633695141, 'logprobs': None, 'model_name': 'llama3.2:latest', 'model_provider': 'ollama'} id='lc_run--019b74d6-48d2-7ff2-ac5b-d181018c4180-0' tool_calls=[{'name': 'WeatherSchema', 'args': {'condition': 'sunny', 'temperature': '75', 'unit': 'fahrenheit'}, 'id': 'ccde391d-b054-4419-9542-3f0d2362aa44', 'type': 'tool_call'}] usage_metadata={'input_tokens': 192, 'output_tokens': 31, 'total_tokens': 223}
================================== Ai Message ==================================
Tool Calls:
  WeatherSchema (ccde391d-b054-4419-9542-3f0d2362aa44)
 Call ID: ccde391d-b054-4419-9542-3f0d2362aa44
  Args:
    condition: sunny
    temperature: 75
    unit: fahrenheit


In [3]:
class SpamSchema(BaseModel):
    classification: str = Field(description="Email classification: spam or not_spam")
    confidence: float = Field(description="Confidence score between 0 and 1")
    reason: str = Field(description="Reason for the classification")

spam_llm = llm.bind_tools(tools=[SpamSchema])
response = spam_llm.invoke("I'm a Nigerian prince, you want to be rich")
# Returns: {"classification": "spam", "confidence": 0.95, "reason": "Nigerian prince scam"}
print(response)
print(response.tool_calls)
response.pretty_print()

content="I can't help with that. Is there anything else I can help you with?" additional_kwargs={} response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-12-31T14:36:10.002862611Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2404270358, 'load_duration': 147654945, 'prompt_eval_count': 198, 'prompt_eval_duration': 803194168, 'eval_count': 18, 'eval_duration': 1412521954, 'logprobs': None, 'model_name': 'llama3.2:latest', 'model_provider': 'ollama'} id='lc_run--019b74d6-6a25-79f3-a3be-b42656b0af0f-0' usage_metadata={'input_tokens': 198, 'output_tokens': 18, 'total_tokens': 216}
[]
================================== Ai Message ==================================

I can't help with that. Is there anything else I can help you with?


In [4]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

# Force tool usage with explicit system instruction
system_prompt = """You are a spam detector. ALWAYS use the SpamSchema tool for every email.
NEVER respond conversationally. RESPOND ONLY with tool call."""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    ("human", "{input}")
])

spam_llm = prompt | llm.bind_tools(tools=[SpamSchema])
response = spam_llm.invoke("I'm a Nigerian prince, you want to be rich")
print(response.tool_calls)
response.pretty_print()

[{'name': 'SpamSchema', 'args': {'classification': 'not_spam', 'confidence': 0, 'reason': 'Lack of specific details and common scam tactic'}, 'id': '4f4d564c-32e0-4cb2-8c74-eaffc6844e74', 'type': 'tool_call'}]
================================== Ai Message ==================================
Tool Calls:
  SpamSchema (4f4d564c-32e0-4cb2-8c74-eaffc6844e74)
 Call ID: 4f4d564c-32e0-4cb2-8c74-eaffc6844e74
  Args:
    classification: not_spam
    confidence: 0
    reason: Lack of specific details and common scam tactic


In [5]:
response.content
response.response_metadata

{'model': 'llama3.2:latest',
 'created_at': '2025-12-31T14:36:14.23944517Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 4142914045,
 'load_duration': 126865446,
 'prompt_eval_count': 226,
 'prompt_eval_duration': 1284317151,
 'eval_count': 32,
 'eval_duration': 2661633490,
 'logprobs': None,
 'model_name': 'llama3.2:latest',
 'model_provider': 'ollama'}

# Defining Reusable Math Tool Schemas

In [6]:
from pydantic import BaseModel
from typing import Literal

class TwoOperands(BaseModel):
    a: float
    b: float

class AddInput(TwoOperands):
    operation: Literal['add']

class SubtractInput(TwoOperands):
    operation: Literal['subtract']

class MultiplyInput(TwoOperands):
    operation: Literal['multiply']

class DivideInput(TwoOperands):
    operation: Literal['divide']

class MathOutput(BaseModel):
    result: float

# Tool Functions Using Pydantic Models

In [7]:
def add_tool(data: AddInput) -> MathOutput:
    return MathOutput(result=data.a + data.b)

def subtract_tool(data: SubtractInput) -> MathOutput:
    return MathOutput(result=data.a - data.b)

def multiply_tool(data: MultiplyInput) -> MathOutput:
    return MathOutput(result=data.a * data.b)
    
def divide_tool(data: DivideInput) -> MathOutput:
    return MathOutput(result=data.a / data.b)

# Dispatching Tool Calls from JSON Input

In [8]:
incoming_json = '{"a": 7, "b": 3, "operation": "subtract"}'

def dispatch_tool(json_payload: str) -> str:
    base = SubtractInput.model_validate_json(json_payload)
    if base.operation == "add":
        output = add_tool(AddInput.model_validate_json(json_payload))
    elif base.operation == "subtract":
        output = subtract_tool(SubtractInput.model_validate_json(json_payload))
    else:
        raise ValueError("Unsupported operation")
    return output.model_dump_json()

result_json = dispatch_tool(incoming_json)
print(result_json)  # {"result": 4.0}

{"result":4.0}


In [9]:
from typing import Union
from pydantic import ValidationError
    
def dispatch_tool(json_payload: str) -> str:
    # Try to determine correct input type
    input_types = {
        "add": AddInput,
        "subtract": SubtractInput,
        "multiply": MultiplyInput,
        "divide": DivideInput
    }
    
    # Parse base first to get operation
    try:
        base = TwoOperands.model_validate_json(json_payload)  #
        operation = json_payload.split('"operation": "')[1].split('"')[0]
    except:
        raise ValueError("Invalid JSON")
    
    if operation not in input_types:
        raise ValueError(f"Unsupported operation: {operation}")
    
    # Fix 2: Call correct tool
    input_model = input_types[operation]
    tools = {
        "add": add_tool,
        "subtract": subtract_tool,
        "multiply": multiply_tool,
        "divide": divide_tool
    }
    
    output = tools[operation](input_model.model_validate_json(json_payload))
    return output.model_dump_json()

incoming_json = '{"a": 12, "b": 3, "operation": "divide"}'

result_json = dispatch_tool(incoming_json)
print(result_json)  # {"result": 4.0}

{"result":4.0}


# Calculator

In [10]:
# Define a schema with Literal to restrict operation types
class CalculatorSchema(BaseModel):
    operation: Literal['add', 'subtract', 'multiply', 'divide'] = Field(
        description="The mathematical operation to perform"
    )
    a: float = Field(description="First number")
    b: float = Field(description="Second number")
    
calculator_llm = llm.bind_tools(tools=[CalculatorSchema])

# ROLE OF Literal

In [11]:
from typing import Literal
import json

# Test with valid operations
response1 = calculator_llm.invoke("Add 15 and 23")
print(response1.tool_calls[0]['args'])
# Output: {"operation": "add", "a": 15.0, "b": 23.0}
result_json1 = dispatch_tool(json.dumps(response1.tool_calls[0]['args']))
print(result_json1)

print('-'*50)

response2 = calculator_llm.invoke("Multiply 7 by 8")
print(response2.tool_calls[0]['args'])
# Output: {"operation": "multiply", "a": 7.0, "b": 8.0}
result_json2 = dispatch_tool(json.dumps(response2.tool_calls[0]['args']))
print(result_json2)
print('-'*50)

{'a': 15, 'b': 23, 'operation': 'add'}
{"result":38.0}
--------------------------------------------------
{'a': 7, 'b': 8, 'operation': 'multiply'}
{"result":56.0}
--------------------------------------------------


In [12]:
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage
from langchain_ollama import OllamaLLM, ChatOllama
from pydantic import BaseModel
from typing import Literal
from typing import Literal
from typing import Union
from pydantic import ValidationError

# Setup LLM
model_name = "qwen2.5:3b" #"llama3.2:latest"  "phi3:mini"
# Instantiate the LLM with the specified model
llm = ChatOllama(
    model=model_name,
    temperature = 3
)

#Defining Reusable Math Tool Schemas
class TwoOperands(BaseModel):
    a: float
    b: float

class AddInput(TwoOperands):
    operation: Literal['add']

class SubtractInput(TwoOperands):
    operation: Literal['subtract']

class MultiplyInput(TwoOperands):
    operation: Literal['multiply']

class DivideInput(TwoOperands):
    operation: Literal['divide']

class MathOutput(BaseModel):
    result: float
    
# Tool Functions Using Pydantic Models
def add_tool(data: AddInput) -> MathOutput:
    return MathOutput(result=data.a + data.b)

def subtract_tool(data: SubtractInput) -> MathOutput:
    return MathOutput(result=data.a - data.b)

def multiply_tool(data: MultiplyInput) -> MathOutput:
    return MathOutput(result=data.a * data.b)
    
def divide_tool(data: DivideInput) -> MathOutput:
    return MathOutput(result=data.a / data.b)

   
def dispatch_tool(json_payload: str) -> str:
    # Try to determine correct input type
    input_types = {
        "add": AddInput,
        "subtract": SubtractInput,
        "multiply": MultiplyInput,
        "divide": DivideInput
    }
    
    # Parse base first to get operation
    try:
        base = TwoOperands.model_validate_json(json_payload)  #
        operation = json_payload.split('"operation": "')[1].split('"')[0]
    except:
        raise ValueError("Invalid JSON")
    
    if operation not in input_types:
        raise ValueError(f"Unsupported operation: {operation}")
    
    # Fix 2: Call correct tool
    input_model = input_types[operation]
    tools = {
        "add": add_tool,
        "subtract": subtract_tool,
        "multiply": multiply_tool,
        "divide": divide_tool
    }
    
    output = tools[operation](input_model.model_validate_json(json_payload))
    return output.model_dump_json()

# incoming_json = '{"a": 12, "b": 3, "operation": "divide"}'

# result_json = dispatch_tool(incoming_json)
# print(result_json)  # {"result": 4.0}

# Define a schema with Literal to restrict operation types
class CalculatorSchema(BaseModel):
    operation: Literal['add', 'subtract', 'multiply', 'divide'] = Field(
        description="The mathematical operation to perform"
    )
    a: float = Field(description="First number")
    b: float = Field(description="Second number")
    
calculator_llm = llm.bind_tools(tools=[CalculatorSchema])

In [13]:
# Define a schema with Literal to restrict operation types
class CalculatorSchema(BaseModel):
    operation: Literal['add', 'subtract', 'multiply', 'divide'] = Field(
        description="The mathematical operation to perform"
    )
    a: float = Field(description="First number")
    b: float = Field(description="Second number")
    
calculator_llm = llm.bind_tools(tools=[CalculatorSchema])


query = "What will be the simple interest for 2 years if Principal = 1000, rate of interest= 4% yearly"
response = calculator_llm.invoke(query)
response.tool_calls

[{'name': 'CalculatorSchema',
  'args': {'a': 1000, 'b': 0.04, 'operation': 'multiply'},
  'id': 'ccb16d56-6ebe-4063-84a8-00424541b5b2',
  'type': 'tool_call'},
 {'name': 'CalculatorSchema',
  'args': {'a': 'result of previous calculation',
   'b': 2,
   'operation': 'multiply'},
  'id': '85f7b8c3-9146-4907-86d6-db86e59ec699',
  'type': 'tool_call'}]

# Calculator Tool Definition

In [14]:
from pydantic import BaseModel, Field, ValidationError
from typing import Literal, Union

OperationType = Literal[
    "add",
    "subtract",
    "multiply",
    "divide",
    "power",          # a ** b
    "modulus",        # a % b
    "floor_divide",   # a // b
    "square_root",    # sqrt(a)  (b is ignored)
    "absolute",       # abs(a)   (b is ignored)
]


class CalculatorInput(BaseModel):
    """Input schema for all calculator operations."""
    operation: OperationType = Field(..., description="The mathematical operation to perform")
    a: float = Field(..., description="First operand (main number)")
    b: float = Field(default=0.0, description="Second operand (used only when required)")

In [15]:
for field_name, field_info in CalculatorInput.model_fields.items():
    print(f"Field: {field_name}")
    print(f"  Type: {field_info.annotation}")
    print(f"  Required: {field_info.is_required()}")
    print(f"  Default: {field_info.default}")
    print(f"  Description: {field_info.description}")
    print(f"  Metadata: {field_info.metadata}")
    print("---")

Field: operation
  Type: typing.Literal['add', 'subtract', 'multiply', 'divide', 'power', 'modulus', 'floor_divide', 'square_root', 'absolute']
  Required: True
  Default: PydanticUndefined
  Description: The mathematical operation to perform
  Metadata: []
---
Field: a
  Type: <class 'float'>
  Required: True
  Default: PydanticUndefined
  Description: First operand (main number)
  Metadata: []
---
Field: b
  Type: <class 'float'>
  Required: False
  Default: 0.0
  Description: Second operand (used only when required)
  Metadata: []
---


In [16]:
class CalculatorOutput(BaseModel):
    """Standardized output format."""
    result: float
    operation_performed: str
    input_used: CalculatorInput

In [17]:
for field_name, field_info in CalculatorOutput.model_fields.items():
    print(f"Field: {field_name}")
    print(f"  Type: {field_info.annotation}")
    print(f"  Required: {field_info.is_required()}")
    print(f"  Default: {field_info.default}")
    print(f"  Description: {field_info.description}")
    print(f"  Metadata: {field_info.metadata}")
    print("---")

Field: result
  Type: <class 'float'>
  Required: True
  Default: PydanticUndefined
  Description: None
  Metadata: []
---
Field: operation_performed
  Type: <class 'str'>
  Required: True
  Default: PydanticUndefined
  Description: None
  Metadata: []
---
Field: input_used
  Type: <class '__main__.CalculatorInput'>
  Required: True
  Default: PydanticUndefined
  Description: None
  Metadata: []
---


# Calculator Engine Class

In [18]:
class MathCalculator:
    """A clean, extensible calculator that handles dispatching based on validated input."""

    @staticmethod
    def _add(a: float, b: float) -> float:
        return a + b

    @staticmethod
    def _subtract(a: float, b: float) -> float:
        return a - b

    @staticmethod
    def _multiply(a: float, b: float) -> float:
        return a * b

    @staticmethod
    def _divide(a: float, b: float) -> float:
        if b == 0:
            raise ValueError("Division by zero is not allowed")
        return a / b

    @staticmethod
    def _power(a: float, b: float) -> float:
        return a ** b

    @staticmethod
    def _modulus(a: float, b: float) -> float:
        if b == 0:
            raise ValueError("Modulus by zero is not allowed")
        return a % b

    @staticmethod
    def _floor_divide(a: float, b: float) -> float:
        if b == 0:
            raise ValueError("Floor division by zero is not allowed")
        return a // b

    @staticmethod
    def _square_root(a: float, _: float) -> float:
        if a < 0:
            raise ValueError("Square root of negative number is not supported in real domain")
        return a ** 0.5

    @staticmethod
    def _absolute(a: float, _: float) -> float:
        return abs(a)

    @classmethod
    def execute(cls, input_data: CalculatorInput) -> CalculatorOutput:
        """Dispatch to the correct operation and return structured output."""
        handlers = {
            "add": cls._add,
            "subtract": cls._subtract,
            "multiply": cls._multiply,
            "divide": cls._divide,
            "power": cls._power,
            "modulus": cls._modulus,
            "floor_divide": cls._floor_divide,
            "square_root": cls._square_root,
            "absolute": cls._absolute,
        }

        handler = handlers[input_data.operation]
        result = handler(input_data.a, input_data.b)

        return CalculatorOutput(
            result=result,
            operation_performed=input_data.operation,
            input_used=input_data,
        )

In [19]:
import inspect

print("Methods in MathCalculator:")
for name, method in inspect.getmembers(MathCalculator, predicate=inspect.isfunction):
    if name.startswith('_') or name == 'execute':
        print(f"{name}{inspect.signature(method)}")

Methods in MathCalculator:
_absolute(a: float, _: float) -> float
_add(a: float, b: float) -> float
_divide(a: float, b: float) -> float
_floor_divide(a: float, b: float) -> float
_modulus(a: float, b: float) -> float
_multiply(a: float, b: float) -> float
_power(a: float, b: float) -> float
_square_root(a: float, _: float) -> float
_subtract(a: float, b: float) -> float


# Calculations

In [20]:
# Bind the single tool schema to the LLM
calculator_llm = llm.bind_tools(tools=[CalculatorInput], tool_choice="auto")

In [21]:
# from __future__ import annotations

# import json
# from typing import Literal, Union

# from pydantic import BaseModel, Field, ValidationError
# from langchain_core.messages import AIMessage


def run_calculation(query: str) -> CalculatorOutput:
    """Run a natural language query through the LLM and execute the calculator."""
    response: AIMessage = calculator_llm.invoke(query)

    if not response.tool_calls:
        raise ValueError("LLM did not call the calculator tool. Response: " + response.content)

    tool_call = response.tool_calls[0]
    args = tool_call["args"]

    # Validate and parse input
    try:
        calc_input = CalculatorInput.model_validate(args)
    except ValidationError as e:
        raise ValueError(f"Invalid arguments from LLM: {e}")

    # Execute the operation
    return MathCalculator.execute(calc_input)

In [23]:
examples = [
    "25+70-80+5",
    "What is 15 plus 27?",
    "Subtract 13 from 50",
    "Multiply 8 by 9",
    "Divide 100 by 4",
    "What is 5 to the power of 3?",
    "What is 17 mod 5?",
    "Floor divide 20 by 6",
    "Square root of 144",
    "Absolute value of -42",

]

for q in examples:
    print(f"\nQuery: {q}")
    try:
        output = run_calculation(q)
        print(f"Result: {output.result}  ({output.operation_performed})")
        print(f"Full output: {output.model_dump_json(indent=2)}")
    except Exception as e:
        print(f"Error: {e}")
    print('-'*100)


Query: 25+70-80+5
Result: 95.0  (add)
Full output: {
  "result": 95.0,
  "operation_performed": "add",
  "input_used": {
    "operation": "add",
    "a": 25.0,
    "b": 70.0
  }
}
----------------------------------------------------------------------------------------------------

Query: What is 15 plus 27?
Result: 42.0  (add)
Full output: {
  "result": 42.0,
  "operation_performed": "add",
  "input_used": {
    "operation": "add",
    "a": 15.0,
    "b": 27.0
  }
}
----------------------------------------------------------------------------------------------------

Query: Subtract 13 from 50
Result: 37.0  (subtract)
Full output: {
  "result": 37.0,
  "operation_performed": "subtract",
  "input_used": {
    "operation": "subtract",
    "a": 50.0,
    "b": 13.0
  }
}
----------------------------------------------------------------------------------------------------

Query: Multiply 8 by 9
Result: 72.0  (multiply)
Full output: {
  "result": 72.0,
  "operation_performed": "multiply",
  